In [61]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from selenium.common.exceptions import NoSuchElementException, ElementClickInterceptedException
from webdriver_manager.chrome import ChromeDriverManager
import time
import mysql.connector
import re

In [62]:
# MySQL connection setup
db = mysql.connector.connect(
    host="127.0.0.1",  # Typically 'localhost' or '127.0.0.1'
    user="root",
    password="kjhyun0221",
    database="news"
)
cursor = db.cursor()

# Create table if it doesn't exist
cursor.execute("""
CREATE TABLE IF NOT EXISTS wti_books (
    id INT AUTO_INCREMENT PRIMARY KEY,
    title VARCHAR(255),
    news_company VARCHAR(127),
    url VARCHAR(255),
    published_date DATE
)
""")

def insert_data(title, news_company, url, published_date):
    # Remove the trailing period and any spaces
    published_date = published_date.strip().rstrip('.')
    
    # Split the date string by periods
    year, month, day = published_date.split('.')
    
    # Remove any leading or trailing spaces that may have been left over
    year = year.strip()
    month = month.strip()
    day = day.strip()
    
    # Construct the MySQL-compatible date string
    mysql_date = f"{year}-{month.zfill(2)}-{day.zfill(2)}"
    
    query = "INSERT INTO wti_books (title, news_company, url, published_date) VALUES (%s, %s, %s, %s)"
    cursor.execute(query, (title, news_company, url, mysql_date))
    db.commit()

In [67]:
chromedriver_path = 'C:/Users/kkksk/.wdm/drivers/chromedriver/win64/122.0.6261.129/chromedriver.exe'
service = Service(chromedriver_path)
driver = webdriver.Chrome(service=service)

In [68]:
target_year = 1986
current_year = 2004

try:
    while target_year <= current_year:
        
        url = f"https://www.google.com/search?q=crude+oil&tbs=bkt:s,cdr:1,cd_min:1/1/{target_year},cd_max:12/31/{target_year}&tbm=bks"
        driver.get(url)
        
        time.sleep(30)
        
        while True:
            news_results = driver.find_elements(By.CSS_SELECTOR, 'div#rso>div')
            if not news_results:  # Check if news_results is empty
                print(f"No results found for the year {target_year}. Moving to the next year.")
                break  # Exit the inner while loop to proceed with the next year
            
            for news_div in news_results:
                try:
                    Link = news_div.find_element(By.TAG_NAME, 'a').get_attribute('href')
                    title_element = news_div.find_element(By.CSS_SELECTOR, 'a>h3')
                    Title = title_element.text.strip(".")
                    Title = re.sub(r"-\d+페이지", "", Title)
                    news_elements = news_div.find_elements(By.CSS_SELECTOR, 'div')
                    company_and_date = [elem.text for elem in news_elements]
                    if len(company_and_date) >= 10:
                        info = company_and_date[10]
                        info_text = info.split("·")
                        Company = info_text[0]
                        Date = info_text[1]

                        insert_data(Title, Company, Link, Date)

                except Exception as e:
                    print(e)
            
            try:
                next_button = driver.find_element(By.ID, 'pnnext')
                next_button.click()
                time.sleep(2)
            except NoSuchElementException:
                break
        
        target_year += 1

except Exception as e:
    print(f"Encountered an exception: {e}")

finally:
    driver.quit()


No results found for the year 2004. Moving to the next year.
